In [1]:
import pdfplumber

def extract_text_with_spaces(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            page_text = ''
            words = []
            previous_char = None

            # Extract character-level details
            for char in page.chars:
                if previous_char:
                    # Calculate space based on character positions
                    space_threshold = 0.5 * (char['fontname'] == previous_char['fontname']) # Adjust this threshold if needed
                    if char['x0'] > previous_char['x1'] + space_threshold:
                        page_text += ' '
                page_text += char['text']
                previous_char = char

            text += page_text + '\n'
    return text

# Use the function to extract text with spaces
pdf_path = 'C:\\MyFolder\\earningscall.pdf'
text = extract_text_with_spaces(pdf_path)

# Print or process the text as needed
print(text)


Coinbase Global, Inc.Second Quarter 2024 Earnings CallAugust 1, 2024Anil Gupta, Vice President, Investor Relations: Good afternoon, and welcome to the Coinbase Second Quarter2024 Earnings Call. Joining me on today's call are Brian Armstrong, Co-Founder and CEO; Emilie Choi, Presidentand COO; Alesia Haas, CFO; and Paul Grewal, Chief Legal Officer.I hope you've all had the opportunity to read our shareholder letter, which was published on our InvestorRelations website earlier today. Before we get started, I'd like to remind you that during today's call we maymake forward-looking statements. Actual results may vary materially from today's statements.Information concerning risks, uncertainties and other factors that could cause these results to differ is includedin our SEC filings. Our discussion today will also include references to certain non-GAAP financial measures.Reconciliations to the most directly comparable GAAP financial measures are provided in our shareholderletter on our Inves

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter



from langchain.text_splitter import RecursiveCharacterTextSplitter

# Example transcript
transcript = text

# Create the text splitter with desired parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # Maximum size of each chunk in characters
    chunk_overlap=20,  # Overlap between chunks to preserve context
    separators=["\n\n", "\n", ".", " "]  # Order of separators to use for splitting
)

# Split the transcript into chunks
chunks = text_splitter.split_text(transcript)

# Output the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")


Chunk 1:
Coinbase Global, Inc.Second Quarter 2024 Earnings CallAugust 1, 2024Anil Gupta, Vice President, Investor Relations: Good afternoon, and welcome to the Coinbase Second Quarter2024 Earnings Call

Chunk 2:
. Joining me on today's call are Brian Armstrong, Co-Founder and CEO; Emilie Choi, Presidentand COO; Alesia Haas, CFO; and Paul Grewal, Chief Legal Officer.I hope you've all had the opportunity to read our shareholder letter, which was published on our InvestorRelations website earlier today

Chunk 3:
. Before we get started, I'd like to remind you that during today's call we maymake forward-looking statements. Actual results may vary materially from today's statements

Chunk 4:
.Information concerning risks, uncertainties and other factors that could cause these results to differ is includedin our SEC filings. Our discussion today will also include references to certain non-GAAP financial measures

Chunk 5:
.Reconciliations to the most directly comparable GAAP financial measur

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb
from openai import OpenAI

# Example transcript
transcript = text  # Replace `text` with your actual transcript

# Create the text splitter with desired parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # Maximum size of each chunk in characters
    chunk_overlap=20,  # Overlap between chunks to preserve context
    separators=["\n\n", "\n", ".", " "]  # Order of separators to use for splitting
)

# Split the transcript into chunks
chunks = text_splitter.split_text(transcript)

# Initialize ChromaDB client and create/get a collection
client = chromadb.Client()
collection_name = "my_collection"
collection = client.get_or_create_collection(name=collection_name)

# Initialize OpenAI client for embeddings
openai_client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# Function to get embeddings
def get_embedding(text, model="nomic-ai/nomic-embed-text-v1.5-GGUF"):
    text = text.replace("\n", " ")
    return openai_client.embeddings.create(input=[text], model=model).data[0].embedding

# Prepare the data structure for document IDs, embeddings, and metadata
document_ids = []
embeddings = []
metadata = []

# Process each chunk, generate its embedding, and prepare for insertion
for i, chunk in enumerate(chunks):
    doc_id = f"chunk_{i+1}"  # Create a document ID based on the chunk number
    document_ids.append(doc_id)
    
    embedding = get_embedding(chunk)  # Get the embedding for the chunk
    embeddings.append(embedding)
    
    metadata.append({"text": chunk})  # Store the original chunk as metadata

# Upload embeddings, document IDs, and metadata to ChromaDB
collection.add(
    ids=document_ids,
    embeddings=embeddings,
    metadatas=metadata
)

print(f"Uploaded {len(embeddings)} embeddings to the '{collection_name}' collection.")


APIConnectionError: Connection error.